In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 1000)
import seaborn as sns
import unidecode

In [2]:
#Lendo dados
drogas_final = pd.read_csv('Dados/GESEG_SBX_DROGA_EXPORT.csv', encoding = 'latin-1', sep = ';', index_col=0)
#Pivotando valor "QUANTIDADE" da coluna "ATRIBUTOS" com o mesmo "NRO_INT_OBJETO"
drogas_final_pivot = drogas_final.pivot(index="NRO_INT_OBJETO", columns="ATRIBUTO", values="QUANTIDADE")
#Agrupando dados com o mesmo "K93_NRO_INT_OBJETO"
#Foi preciso utilizar "agg('min')" pois apenas groupby()...
#retorna um objeto iterável 
dg2_final = drogas_final[['NRO_INT_OBJETO','ANO_OCOR', 'NRO_OCOR','NRO_INT_TIPO_OBJETO','TIPO_OBJETO','ATRIBUTO']].groupby(["NRO_INT_OBJETO"]).min()
#Ordenando por "K93_NRO_INT_OBJETO"
dg2_final_sorted = dg2_final.sort_values(by=['NRO_INT_OBJETO'])
drogas_pivot_final_sorted = drogas_final_pivot.sort_values(by=['NRO_INT_OBJETO'])

In [3]:
#Concatenando as duas tabelas
try_1_final = pd.concat([dg2_final_sorted, drogas_pivot_final_sorted],axis=1)
#Atribuindo a tabela resultado "try_1" com um index numérico
resultado = try_1_final.reset_index()
#Renomeando Coluans
resultado = resultado.rename(columns={"NRO_INT_OBJETO": "CHAVE_OBJETO", "TIPO_OBJETO": "TXT_TIPO_OBJETO"})

In [4]:
def ajusta_texto_numerico(texto):
    return str(texto).replace('.', '').replace(',', '.')

resultado['Peso Unitário'] = resultado['Peso Unitário'].apply(ajusta_texto_numerico)
resultado['Peso Total'] = resultado['Peso Total'].apply(ajusta_texto_numerico)

resultado['Peso Unitário'] = pd.to_numeric(resultado['Peso Unitário'], errors='coerce',downcast="float").fillna(0).astype(float)
resultado['Peso Total'] = pd.to_numeric(resultado['Peso Total'], errors='coerce',downcast="float").fillna(0).astype(float)

In [5]:
#Definindo uma series Drogas
#Como resultado do tipo de objeto de resultado = Drogas
Drogas = resultado.loc[resultado['TXT_TIPO_OBJETO'] == 'Drogas']

Drogas = resultado.loc[resultado['TXT_TIPO_OBJETO'] == 'Drogas']
Outras = resultado.loc[resultado['TXT_TIPO_OBJETO'] != 'Drogas']
Outras = Outras.reset_index(drop= True)

#DF com descrição nula
DrogasNA = Drogas.loc[Drogas['Descrição'].isnull()]
DrogasNA = DrogasNA.reset_index(drop=True)

#Excluindo descrição nula do DF Drogas
Drogas = Drogas.loc[Drogas['Descrição'].notna()]
Drogas = Drogas.reset_index(drop=True)

In [6]:
def changer(txt):
    return unidecode.unidecode(str(txt).replace(',','').replace('.',''))

Drogas['Descrição'] = Drogas['Descrição'].apply(changer)
    
Drogas['Descrição'] = Drogas['Descrição'].str.lower()

In [7]:
#Tratando ajustes de drogas
def analizer(x):
    
    if 'maconha' in x:
        return 'Maconha'
    elif 'canabis' in x:
        return 'Maconha'
    elif 'cannabis' in x:
        return 'Maconha'
    elif 'crack' in x:
        return 'Crack'
    elif 'cocaina' in x:
        return 'Cocaína'
    else:
        return 'Drogas'
    
Drogas['RECLASSIFICAÇÃO'] = Drogas['Descrição'].apply(analizer)

In [8]:
listaDrogas = [Drogas, DrogasNA]
Drogas = pd.concat(listaDrogas)

stores_df = Drogas

In [9]:
#Concatenando DF Drogas com DF outros
resultado_final = pd.concat([Drogas, Outras])

#Tratando NaN de reclassificação
resultado_final['RECLASSIFICAÇÃO'] = resultado_final['RECLASSIFICAÇÃO'].fillna(value=resultado_final.TXT_TIPO_OBJETO)

#Definindo a chave do objeto como indice
resultado_final = resultado_final.set_index('CHAVE_OBJETO')

In [10]:
resultado_final['Unidade de Peso'] = resultado_final['Unidade de Peso'].apply(str)

In [11]:
def refactor(x,y):
    if 'Miligrama' in x:
        return y/1000
    if 'Quilograma' in x:
        return y * 100
    if 'Decigrama' in x:
        return y / 10
    if 'Tonelada' in x:
        return y * 1000000
    else:
        return y
        

#df['newcolumn'] = df.apply(lambda x: fxy(x['A'], x['B']), axis=1)
resultado_final['Peso Unitário Rec'] = resultado_final.apply(lambda x: refactor(x['Unidade de Peso'], x['Peso Unitário']),axis=1)
#resultado_final['Peso Unitário Rec'] = resultado_final[['Unidade de Peso','Peso Unitário']].apply(refactor)



In [12]:
resultado_final.loc[resultado_final["Unidade de Peso"] == "Miligrama"]

,ANO_OCOR,NRO_OCOR,NRO_INT_TIPO_OBJETO,TXT_TIPO_OBJETO,ATRIBUTO,Cor Predominante,Descrição,Peso Total,Peso Unitário,Quantidade de Unidade(s),Tipo de Embalagem,Unidade de Peso,RECLASSIFICAÇÃO,Peso Unitário Rec
CHAVE_OBJETO,,,,,,,,,,,,,,
19903932,2018,8070,1025,Drogas,Cor Predominante,Verde,uma pequena porcao de erva de cor esverdeada s...,0.00,0.00,1,Porção(es),Miligrama,Maconha,0.00000
20064028,2018,4078,1025,Drogas,Cor Predominante,Branco,2g de cocaina em 2 petecas de um grama cada,0.20,0.10,2,Porção(es),Miligrama,Cocaína,0.00010
20181002,2018,12329,1025,Drogas,Cor Predominante,Amarelo,pedras de cor amarelada,0.05,0.05,1,Porção(es),Miligrama,Drogas,0.00005
20197844,2018,12487,1025,Drogas,Cor Predominante,Azul,com embalagem,0.50,0.50,1,Unidade(s),Miligrama,Drogas,0.00050
20198433,2018,33932,1025,Drogas,Cor Predominante,Preto,uma pedrinha de crack embalagem plastica preta,0.10,0.10,1,Unidade(s),Miligrama,Crack,0.00010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24071043,2021,5038,757,Crack,Cor Predominante,Amarelo,UMA PEDRA AMARELA SEMELHANTE A CRACK,0.01,0.01,1,Porção(es),Miligrama,Crack,0.00001
24084828,2021,1704,5,Maconha,Cor Predominante,Verde,Cigarro parcialmente consumido,0.00,0.00,01,Cigarro(s),Miligrama,Maconha,0.00000
24098991,2021,345,5,Maconha,Cor Predominante,Verde,UMA PORÇÃO DE SUBSTÂNICA ESVERDEADA SEMLEHANTE...,0.00,0.00,2000,Porção(es),Miligrama,Maconha,0.00000
